In [1]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

import cv2 as cv

# Import IPLabsImage() Class
import sys  
from iplabs import IPLabViewer as viewer

# Load images to be used in this lab 
plate        = cv.imread('images/plate.tif',               cv.IMREAD_UNCHANGED)

In [2]:
%use javascript
// import IPLabImageAccess.js as Image
var Image = require('./IPLabImageAccess.js')

In [30]:
var a = new Array(10);
console.log(a)
for(var i=0; i<10; i++){
    a[i] = i;
}

var b = [a]
var i = new Image(b)
i

[ <10 empty items> ]


IPLabImageAccess {
  image: [ [
      0, 1, 2, 3, 4,
      5, 6, 7, 8, 9
    ] ],
  nx: 10,
  ny: 1,
  min: 0,
  max: 9,
  change_check: false
}

In [9]:
var t = new Image(5, 5);
for(var i=0; i<t.nx; i++){
    for(var j=0; j<t.ny; j++){
        t.setPixel(i, j, j*t.ny+i)
    }
}
console.log(t.visualize())
var nbh = t.getNbh(2, 2, 11, 11);
var vis = new Image(nbh.ny, nbh.nx);
for(var i=0; i<nbh.nx; i++){
    for(var j=0; j<nbh.ny; j++){
        vis.setPixel(i, j, nbh.getPixel(i, j))
    }
}
console.log(vis.visualize())
console.log(nbh.visualize())

[[  0  1  2  3  4 ]
 [  5  6  7  8  9 ]
 [ 10 11 12 13 14 ]
 [ 15 16 17 18 19 ]
 [ 20 21 22 23 24 ]]

[[ 12 11 10 10 11 12 13 14 14 13 12 ]
 [  7  6  5  5  6  7  8  9  9  8  7 ]
 [  2  1  0  0  1  2  3  4  4  3  2 ]
 [  2  1  0  0  1  2  3  4  4  3  2 ]
 [  7  6  5  5  6  7  8  9  9  8  7 ]
 [ 12 11 10 10 11 12 13 14 14 13 12 ]
 [ 17 16 15 15 16 17 18 19 19 18 17 ]
 [ 22 21 20 20 21 22 23 24 24 23 22 ]
 [ 22 21 20 20 21 22 23 24 24 23 22 ]
 [ 17 16 15 15 16 17 18 19 19 18 17 ]
 [ 12 11 10 10 11 12 13 14 14 13 12 ]]

[[ 12 11 10 10 11 12 13 14 14 13 12 ]
 [  7  6  5  5  6  7  8  9  9  8  7 ]
 [  2  1  0  0  1  2  3  4  4  3  2 ]
 [  2  1  0  0  1  2  3  4  4  3  2 ]
 [  7  6  5  5  6  7  8  9  9  8  7 ]
 [ 12 11 10 10 11 12 13 14 14 13 12 ]
 [ 17 16 15 15 16 17 18 19 19 18 17 ]
 [ 22 21 20 20 21 22 23 24 24 23 22 ]
 [ 22 21 20 20 21 22 23 24 24 23 22 ]
 [ 17 16 15 15 16 17 18 19 19 18 17 ]
 [ 12 11 10 10 11 12 13 14 14 13 12 ]]



In [4]:
function erode(img, b){ 
    var output = new Image(img.shape());
    // loop through every pixel of the image
    for(var x = 0; x < img.nx; x++){
        for(var y = 0; y < img.ny; y++){
            // extract the neighbourhood around pixel (x,y)
            var nbh = img.getNbh(x, y, b.nx, b.ny);
            var valmin = Number.MAX_VALUE;
            // loop through every pixel of the neighbourhood
            for(var k = 0; k < b.ny; k++){
                for(var l = 0; l < b.nx; l++){
                    if (b.getPixel(l, k)){
                        // perform erosion (calculate new minimum value)
                        valmin = Math.min(nbh.getPixel(l, k), valmin);
                    }
                }
            }            
            // Set the eroded pixel value in the output image
            output.setPixel(x, y, valmin);
        }
    }
    return output;
}

In [5]:
function erode_raw(img, b){
    // initialize the output array with the first dimension
    var output = new Array(img.length);
    // for every element of the first dimension, add the second dimension
    for(var i=0; i < output.length; i++){
        // create the second dimension
        output[i] = new Array(img[i].length);
        for(var j=0; j < output[i].length; j++){
            // initialize output value to 0
            output[i][j] = 0;
        }
    }
    // loop through every pixel of the image
    for(var y = 0; y < img.length; y++){
        for(var x = 0; x < img[y].length; x++){
            var valmin = Number.MAX_VALUE;
            // loop through every pixel of the neighbourhood
            for(var k = 0; k < b.length; k++){
                for(var l = 0; l < b[k].length; l++){
                    if (b[k][l]){
                        var y_ = y - parseInt(b.length/2) + k;
                        var x_ = x - parseInt(b[k].length/2) + l;
                        // Apply boundary conditions
                        if(y_ < 0){
                            // apply mirror folding
                            y_ = -y_ - 1;
                        }
                        if(y_ >= img.length){
                            // apply mirror folding
                            y_ = img.length - 1 - (y_ % img.length);
                        }
                        if(x_ < 0){
                            // apply mirror folding
                            x_ = -x_ - 1;
                        }if(x_ >= img[y].length){
                            // apply mirror folding
                            x_ = img[y].length - 1 - (x_ % img[y].length);
                        }
                        // perform erosion (calculate new minimum value)
                        valmin = Math.min(img[y_][x_], valmin);
                    }
                }
            }
            // Set the eroded pixel value in the output image
            output[y][x] = valmin;
        }
    }
    return output;
}

In [6]:
%use javascript
%get plate
%put t_raw t_normal ns

var ns = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25];
var t_normal = [];
var t_raw = [];

for(var i=0; i<ns.length; i++){
    var t_n = 0;
    var t_i = 0;
    var t_r = 0;
    for(var k=0; k<3; k++){
        var n = ns[i]
        var b_square = new Image(n, n, {'init_value':1});
        var start = new Date();
        var plate_eroded = erode(new Image(plate), b_square).toArray();
        var t1 = new Date();
        var plate_eroded_raw = erode_raw(plate, b_square.toArray());
        var end = new Date();
        t_n += (t1 - start)/3;
        t_r += (end - t1)/3;
    }
    t_raw.push(t_r);
    t_normal.push(t_n);
}

12

In [12]:
plt.close('all')
plt.figure()
plt.plot(ns, t_raw, label='raw')
plt.plot(ns, t_normal, label='normal')
plt.legend()
plt.title('Runtime of different erode functions.')
plt.xlabel('Kernel size')
plt.ylabel('Runtime [ms]')
plt.show()
plt.figure()
plt.plot(np.array(t_normal)/np.array(t_raw))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

In [13]:
%use javascript
%put plate_eroded plate_eroded_raw

var n = 7
var b_square = new Image(n, n, {'init_value':1});
var plate_eroded = erode(new Image(plate), b_square).toArray();
var plate_eroded_raw = erode_raw(plate, b_square.toArray());

In [14]:
plt.close('all')
view = viewer([np.array(plate_eroded), np.array(plate_eroded_raw)], subplots=(1,2))
np.testing.assert_array_equal(plate_eroded, plate_eroded_raw)

Button(description='Show Widgets', style=ButtonStyle())

In [70]:
var plate_img = new Image(plate);
var a = 0;

var start = new Date();
for(var i=0; i<plate_img.nx; i++){
    for(var j=0; j<plate_img.ny; j++){
        a = plate_img.getNbh(i, j, 11, 11);
        /*
        // loop through every pixel of the neighbourhood
        for(var k = 0; k < 11; k++){
            for(var l = 0; l < 11; l++){
                var y_ = i - parseInt(11/2) + k;
                var x_ = j - parseInt(11/2) + l;
                // Apply boundary conditions
                if(y_ < 0){
                    // apply mirror folding
                    y_ = -y_ - 1;
                }
                if(y_ >= plate.ny){
                    // apply mirror folding
                    y_ = plate.ny - 1 - (y_ % plate.ny);
                }
                if(x_ < 0){
                    // apply mirror folding
                    x_ = -x_ - 1;
                }if(x_ >= plate.nx){
                    // apply mirror folding
                    x_ = plate.nx - 1 - (x_ % plate.nx);
                }
                // perform erosion (calculate new minimum value)
                a = plate_img.getPixel(x_, y_);
            }
        }
        */
    }
}
var t1 = new Date();
for(var i=0; i<plate.length; i++){
    for(var j=0; j<plate[i].length; j++){
        // loop through every pixel of the neighbourhood
        for(var k = 0; k < 11; k++){
            for(var l = 0; l < 11; l++){
                var y_ = i - parseInt(11/2) + k;
                var x_ = j - parseInt(11/2) + l;
                // Apply boundary conditions
                if(y_ < 0){
                    // apply mirror folding
                    y_ = -y_ - 1;
                }
                if(y_ >= plate.length){
                    // apply mirror folding
                    y_ = plate.length - 1 - (y_ % plate.length);
                }
                if(x_ < 0){
                    // apply mirror folding
                    x_ = -x_ - 1;
                }if(x_ >= plate[i].length){
                    // apply mirror folding
                    x_ = plate[i].length - 1 - (x_ % plate[i].length);
                }
                // perform erosion (calculate new minimum value)
                a = plate[y_][x_];
            }
        }
    }
}
var end = new Date();
console.log(end - t1, t1 - start)

328 560


In [13]:
%use sos
np.testing.assert_array_equal(plate_eroded, plate_eroded_raw)
plt.close('all')
view = viewer([plate, np.array(plate_eroded), np.array(plate_eroded_raw), np.array(plate_eroded) - np.array(plate_eroded_raw)], subplots=(2,2))

Button(description='Show Widgets', style=ButtonStyle())

In [60]:
%use sos

vis_test = np.random.randint(0, 255, size=(5, 10))

In [69]:
%use javascript
%get vis_test

var vis_test_img = new Image(5, 11);
var row = new Image(1, 11, {'init_value':255});
var col = new Image(5, 1, {'init_value':5.3863724772837});
vis_test_img.putRow(2, row);
vis_test_img.putColumn(5, col);
console.log(vis_test_img.image);
console.log(vis_test_img.visualize());

[
  [ 0, 0, 0, 0, 0, 5.3863724772837, 0, 0, 0, 0, 0 ],
  [ 0, 0, 0, 0, 0, 5.3863724772837, 0, 0, 0, 0, 0 ],
  [ 255, 255, 255, 255, 255, 5.3863724772837, 255, 255, 255, 255, 255 ],
  [ 0, 0, 0, 0, 0, 5.3863724772837, 0, 0, 0, 0, 0 ],
  [ 0, 0, 0, 0, 0, 5.3863724772837, 0, 0, 0, 0, 0 ]
]
[[   0.000   0.000   0.000   0.000   0.000   5.386   0.000   0.000   0.000   0.000   0.000 ]
 [   0.000   0.000   0.000   0.000   0.000   5.386   0.000   0.000   0.000   0.000   0.000 ]
 [ 255.000 255.000 255.000 255.000 255.000   5.386 255.000 255.000 255.000 255.000 255.000 ]
 [   0.000   0.000   0.000   0.000   0.000   5.386   0.000   0.000   0.000   0.000   0.000 ]
 [   0.000   0.000   0.000   0.000   0.000   5.386   0.000   0.000   0.000   0.000   0.000 ]]

